<a href="https://colab.research.google.com/github/Beepeen78/pysparkproject/blob/main/ipldatanalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=129b1923a8a6d3341d5c57cad3653ca94d39b2103f55c02e38ad3d6fc9c0a2ca
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('ipl_2022_dataset.csv')

In [4]:
import pandas as pd

In [5]:
from pyspark.sql import SparkSession

In [6]:
from pyspark.sql import *

In [7]:
#start spark session

spark = SparkSession.builder \
        .master("local") \
        .appName("practicesession")\
        .getOrCreate()

In [8]:
df = spark.read.csv('/content/ipl_2022_dataset.csv',inferSchema=True,header=True)

In [9]:
df.show()

+---+-------------------+----------+------------+---------------+---------------+----------+--------------+
|_c0|             Player|Base Price|        TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+-------------------+----------+------------+---------------+---------------+----------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|           10.0|         1300.0|       KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|            9.0|         1170.0|        RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|            8.0|         1040.0|       KKR|Gujarat Titans|
|  5|     Mohammad Shami|      2 Cr|      BOWLER|           6.25|          812.5|      PBKS|Gujarat Titans|
|  6|         Yash Dayal|   

In [52]:
df2 = df.withColumnRenamed("player","name")\
      .withColumnRenamed("Base Price","Base_Price")\
      .withColumnRenamed("TYPE","Speciality")\
      .withColumnRenamed("COST IN Rs (CR.)","INR Price")\
      .withColumnRenamed("Cost IN $ (000)","USD_PRICE")\
      .withColumnRenamed("2021 Squad","2021 Squard")\
      .withColumnRenamed("Team","player")

In [12]:
df2.show()

+---+-------------------+----------+------------+---------------+---------+-----------+--------------+
|_c0|               name|Base Price|  Speciality|COST IN ₹ (CR.)|USD_PRICE|2021 Squard|        player|
+---+-------------------+----------+------------+---------------+---------+-----------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|           15.0|   1950.0|        SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|           15.0|   1950.0|         MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|           10.0|   1300.0|        KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|            9.0|   1170.0|         RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|            8.0|   1040.0|        KKR|Gujarat Titans|
|  5|     Mohammad Shami|      2 Cr|      BOWLER|           6.25|    812.5|       PBKS|Gujarat Titans|
|  6|         Yash Dayal|   20 Lakh|      BOWLER|            3.2|    416.

In [53]:
df3=df2.drop('COST IN ₹ (CR.)')

In [54]:
df3.show()

+---+-------------------+----------+------------+---------+-----------+--------------+
|_c0|               name|Base_Price|  Speciality|USD_PRICE|2021 Squard|        player|
+---+-------------------+----------+------------+---------+-----------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|   1950.0|        SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|   1950.0|         MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|   1300.0|        KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|   1170.0|         RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|   1040.0|        KKR|Gujarat Titans|
|  5|     Mohammad Shami|      2 Cr|      BOWLER|    812.5|       PBKS|Gujarat Titans|
|  6|         Yash Dayal|   20 Lakh|      BOWLER|    416.0|       NULL|Gujarat Titans|
|  7|       David Miller|      1 Cr|      BATTER|    390.0|         RR|Gujarat Titans|
|  8|     R. Sai Kishore|   20 Lakh|      B

In [17]:
#Top Batsman
df3.select('name','USD_PRICE').where(df3.Speciality=='BATTER').orderBy('USD_PRICE', ascending= False).show(3)

+---------------+---------+
|           name|USD_PRICE|
+---------------+---------+
|   Rohit Sharma|   2080.0|
|    Virat Kohli|   1950.0|
|Kane Williamson|   1820.0|
+---------------+---------+
only showing top 3 rows



In [56]:
df3.createOrReplaceTempView('ipl')

In [57]:
# Top 3 batsman who got paid the most?
spark.sql("""SELECT name, USD_PRICE FROM ipl WHERE speciality = 'BATTER' ORDER BY 2 DESC LIMIT 3""").show()

+---------------+---------+
|           name|USD_PRICE|
+---------------+---------+
|   Rohit Sharma|   2080.0|
|    Virat Kohli|   1950.0|
|Kane Williamson|   1820.0|
+---------------+---------+



In [37]:
# Top 5 bowlers who get paid the most?
df3.select('name','USD_PRICE').filter(df3.Speciality=='BOWLER').orderBy('USD_PRICE',ascending=False).show(5)

+---------------+---------+
|           name|USD_PRICE|
+---------------+---------+
|    Rashid Khan|   1950.0|
|  Deepak Chahar|   1820.0|
| Jasprit Bumrah|   1560.0|
| Shardul Thakur|   1397.5|
|Lockie Ferguson|   1300.0|
+---------------+---------+
only showing top 5 rows



In [39]:
spark.sql("""SELECT name, USD_PRICE FROM ipl WHERE speciality = 'BOWLER' ORDER By 2 DESC LIMIT 5 """).show()

+--------------+---------+
|          name|USD_PRICE|
+--------------+---------+
|   Rashid Khan|   1950.0|
| Deepak Chahar|   1820.0|
|Jasprit Bumrah|   1560.0|
|Shardul Thakur|   1397.5|
|    Avesh Khan|   1300.0|
+--------------+---------+



In [42]:
# Top 5 All-Rounders who get paid the most?
spark.sql("""SELECT name, USD_PRICE FROM ipl where speciality='ALL-ROUNDER' ORDER By 2 DESC LIMIT 5""").show()

+----------------+---------+
|            name|USD_PRICE|
+----------------+---------+
| Ravindra Jadeja|   2080.0|
|   Hardik Pandya|   1950.0|
|   Andre Russell|   1560.0|
|Liam Livingstone|   1495.0|
|   Glenn Maxwell|   1430.0|
+----------------+---------+



In [46]:
# Average pay for Batsman, Bowler, All-Rounder, Wicket-Keeper?
from pyspark.sql import functions as F
spark.sql("""SELECT speciality, round(avg(USD_PRICE),2) average_price FROM ipl GROUP BY 1 ORDER BY 2 DESC""").show()


+------------+-------------+
|  speciality|average_price|
+------------+-------------+
|WICKETKEEPER|       661.31|
|      BATTER|       534.01|
| ALL-ROUNDER|       468.81|
|      BOWLER|       399.03|
+------------+-------------+



In [58]:
# List of Retained players with team name and salary?
spark.sql("""SELECT name, USD_PRICE FROM ipl WHERE Base_price ="Retained" ORDER BY 2 DESC LIMIT 5 """).show()

+---------------+---------+
|           name|USD_PRICE|
+---------------+---------+
|Ravindra Jadeja|   2080.0|
|   Rishabh Pant|   2080.0|
|   Rohit Sharma|   2080.0|
|    Virat Kohli|   1950.0|
|   Sanju Samson|   1820.0|
+---------------+---------+



In [60]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
!git clone https://github.com/Beepeen78/Data-analytics-project-with-pyspark.git

fatal: destination path 'Data-analytics-project-with-pyspark' already exists and is not an empty directory.
